In [3]:
#https://github.com/Arakkun/LolWorldsPredictions/blob/main/Dataset-Scrape-And-Analysis.ipynb

# API LOL MATCH STATISTICS

##

# API query from Leaguepedia 

In [4]:
#https://lol.fandom.com/wiki/Help:Leaguepedia_API
""" 
    This is the website I used to figure out how to query API from Leaguepedia (storing all match statistics)
"""
import mwclient

#This is the LeaguePedia website that stores all match and team statistics
site = mwclient.Site('lol.fandom.com', path='/')

##

# Querying match statistics

In [5]:

"""
    I searched for a quicker way to iterate for loops because it took too long to run
"""
#https://github.com/tqdm/tqdm 
from tqdm.notebook import tqdm

#pre-set the leagues I want to look at
leagues=['GPL','IWCQ','WCS','LCS','LEC','LCK','LPL','CBLOL','LCL','LJL','LLA','OPL','PCS','VCS','TCL','LMS','LST','NA LCS','EU LCS','LLN','CLS']

#create an empty list to store my dataset
results = []

#for loop to create my data base of match stats
for league in tqdm(leagues):
    off=0
    
    #while loop to retrive data thruogh API
    while True:
        #API query
        response = site.api('cargoquery',
                            offset=str(off),
                            
                            limit="max",
                            
                            #retrive tables
                            tables = "Tournaments=T, Leagues=L, MatchSchedule=M",
                            
                            #retrive fields as columns
                            fields="T.Name=Tournament, L.League=League, L.League_Short=League_short, M.DateTime_UTC=Date, M.Team1Final=Team1, M.Team2Final=Team2, M.Team1Score, M.Team2Score, M.Winner", 
                            
                            #set condition (only include specified leagues)
                            where = 'L.League_Short = "'+league+'"',
                            
                            #similar to vlookup in the web pages
                            join_on = "T.League = L.League, T.OverviewPage=M.OverviewPage")
        
        #add to database
        results += response["cargoquery"] 
        
        #speed
        off=off+500
        if(len(response["cargoquery"])<500): break

  0%|          | 0/21 [00:00<?, ?it/s]

In [6]:
#I broke this into two parts because the location for the queries were different

In [7]:
off=0
while(True):
        response = site.api('cargoquery',
                            offset=str(off),
                            limit="500",
                            tables = "Tournaments=T, Leagues=L, MatchSchedule=M",
                            fields="T.League, T.Name=Tournament, M.DateTime_UTC=Date, M.Team1Final=Team1, M.Team2Final=Team2, M.Team1Score, M.Team2Score",
                            where = 'T.League = "League of Legends SEA Tour"',
                            join_on = "T.League = L.League, T.OverviewPage=M.OverviewPage")
        results += response["cargoquery"] 
        off=off+500
        if(len(response["cargoquery"])<500): break

##

# Convert to Pandas Dataframe from a json dictionary

In [8]:
import pandas as pd
MatchData = pd.DataFrame ( [ a['title'] for a in results ] )
MatchData.head()

,Tournament,League,League_short,Date,Team1,Team2,Team1Score,Team2Score,Winner,Date__precision
0,GPL 2012 Opening Event,Garena Premier League,GPL,2012-05-06 07:59:00,Bangkok Titans,Saigon Jokers,0,1,2,0
1,GPL 2012 Opening Event,Garena Premier League,GPL,2012-05-06 07:59:00,Kuala Lumpur Hunters,Saigon Jokers,0,1,2,0
2,GPL 2012 Opening Event,Garena Premier League,GPL,2012-05-06 07:59:00,Manila Eagles,Kuala Lumpur Hunters,0,1,2,0
3,GPL 2012 Opening Event,Garena Premier League,GPL,2012-05-06 07:59:00,Saigon Jokers,Singapore Sentinels,0,1,2,0
4,GPL 2012 Opening Event,Garena Premier League,GPL,2012-05-06 07:59:00,Singapore Sentinels,Kuala Lumpur Hunters,1,0,1,0


## clean data

In [9]:
#remove date_precision column - not needed
MatchData = MatchData.drop("Date__precision", axis=1)

#clean NA
MatchData = MatchData.replace("",float("NaN")).dropna(axis='index', how='any')

#sort values by dates and not leagues
MatchData.sort_values(by="Date",inplace=True)

#convert Team 1 & 2 score as integers
MatchData = MatchData.astype({'Team1Score':'int','Team2Score':'int'})

#beautify dates and time 
MatchData["Date"] = pd.to_datetime(MatchData["Date"], format='%Y-%m-%d %H:%M:%S')

MatchData.to_csv("Match_Stats_API_new.csv")

In [10]:
#MatchData.League[MatchData.League=='League of Legends SEA Tour']="LST"

In [11]:
#create dictionary to match start and end of league season
#season dates found on Leaguepedia
dict_match_season = {
    2023:{'Start':'2022-11-11 00:00:00','End':'2023-10-01 00:00:00'},
    2022:{'Start':'2021-11-11 00:00:00','End':'2022-10-01 00:00:00'},  
    2021:{'Start':'2020-11-11 00:00:00','End':'2021-10-01 00:00:00'},  
    2020:{'Start':'2019-11-11 00:00:00','End':'2020-10-01 00:00:00'},  
    2019:{'Start':'2018-11-04 00:00:00','End':'2019-10-09 00:00:00'},
    2018:{'Start':'2017-11-05 00:00:00','End':'2018-10-08 00:00:00'},
    2017:{'Start':'2016-10-30 00:00:00','End':'2017-09-30 00:00:00'}
}

In [12]:
#create dictionary for all match stats within the season 
dict_match_data_per_season=dict()
for date in dict_match_season:
    dict_match_data_per_season[date] = MatchData[
        (MatchData.Date>dict_match_season[date]['Start'])&
        (MatchData.Date<dict_match_season[date]['End'])]

In [13]:
#get match data per season
dict_match_data_per_season[2022]
df_match_data_per_season = pd.DataFrame(dict_match_data_per_season[2022])
df_match_data_per_season = pd.DataFrame(dict_match_data_per_season[2022])
df_match_data_per_season.to_csv("2022 match data.csv")

In [14]:
#get leagues - without duplicates
dict_match_data_per_season[2022]["League_short"].unique()

array(['VCS', 'LPL', 'LCK', 'LEC', 'LCS', 'TCL', 'CBLOL', 'LLA', 'LJL',
       'PCS', 'LCL', 'WCS'], dtype=object)

In [15]:
#because LoL changed the league names, I create a dictionary to match the old and new league names
#alias = {
dict_league_names = {
    'GPL': 'PCS', 
    'NA LCS': 'LCS', 
    'EU LCS': 'LEC', 
    'LPL': 'LPL', 
    'TCL': 'PCS', 
    'LJL': 'LJL', 
    'CBLOL': 'CBLOL',
    'LMS': 'PCS', 
    'OPL': 'OPL', 
    'CLS': 'LLA', 
    'LCK': 'LCK', 
    'LCL': 'LCL', 
    'LLN': 'LLA', 
    'VCS': 'PCS', 
    'LEC': 'LEC',
    'LLA': 'LLA', 
    'LCS': 'LCS', 
    'PCS': 'PCS',
    'LST': 'PCS'
}
list_regions = list(set([dict_league_names[name] for name in dict_league_names]))

In [16]:
dict_regions = {list_regions[i]:i for i in range(len(list_regions))}
dict_regions_indexes = {region:dict_regions[dict_league_names[region]] for region in dict_league_names}
dict_regions

{'LJL': 0,
 'CBLOL': 1,
 'LCL': 2,
 'PCS': 3,
 'OPL': 4,
 'LLA': 5,
 'LEC': 6,
 'LCS': 7,
 'LPL': 8,
 'LCK': 9}

In [17]:
#create a dictionary for Worlds Stats 
dict_Worlds_data=dict()

#create a dictionary for Groups Data
dict_groups_data=dict()

#for all years in stats 2017 - 2022:
for year in range (2017, 2023):
    
    response = site.api('cargoquery',
                        limit="max",
                        tables = "MatchSchedule=M",
                        fields="M.Team1Final=Team1, M.Team2Final=Team2, M.Team1Score, M.Team2Score, M.Tab",
                        where = 'M.OverviewPage = "'+str(year)+' Season World Championship/Main Event"')
    
    #create dictionary for LoL Worlds Championship match stats
    dict_Worlds_data[year] = pd.DataFrame ( [ a['title'] for a in response['cargoquery'] ] )
    
    response = site.api('cargoquery',
                        limit="max",
                        tables = "TournamentGroups=T",
                        fields="T.Team, T.GroupName",
                        where = 'T.OverviewPage = "'+str(year)+' Season World Championship/Main Event"')
    
    #create dictionary for Groups Data
    dict_groups_data[year] = pd.DataFrame ( [ a['title'] for a in response['cargoquery'] ] )

In [18]:
dict_Worlds_data[2022]

,Team1,Team2,Team1Score,Team2Score,Tab
0,100 Thieves,CTBC Flying Oyster,1,0,Day 8
1,100 Thieves,Gen.G,0,1,Day 3
2,100 Thieves,Royal Never Give Up,0,1,Day 4
3,Cloud9,EDward Gaming,0,1,Day 5
4,Cloud9,Fnatic,0,1,Day 1
5,Cloud9,T1,0,1,Day 3
6,CTBC Flying Oyster,100 Thieves,1,0,Day 1
7,CTBC Flying Oyster,Gen.G,0,1,Day 8
8,CTBC Flying Oyster,Royal Never Give Up,0,1,Day 8
9,DRX,EDward Gaming,3,2,Quarterfinals


In [19]:
def WhoWon(row):
    if row["Team1Score"]>row["Team2Score"]: return row["Team1"]
    else: return row["Team2"]

In [22]:
FinalPlayOff = dict() # Will contain the matches for the final play-off
for year in [2017,2018,2019,2020]:
    dict_groups_data[year]["Wins"] = 0
    FinalPlayOff[year] = pd.DataFrame([], columns=dict_Worlds_data[year].columns)
    for index,row in dict_Worlds_data[year].iterrows():
        if (row.Tab not in ['Quarterfinals', 'Semifinals', 'Finals']):
            winner_row = dict_groups_data[year].Team == WhoWon(row)
            dict_groups_data[year].Wins[winner_row] += 1
        else:
            FinalPlayOff[year] = FinalPlayOff[year].append(row)
    FinalPlayOff[year]["Winner"] = FinalPlayOff[year].apply(WhoWon,axis=1)

C:\Users\vivia\AppData\Local\Temp\ipykernel_5112\2708045039.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_groups_data[year].Wins[winner_row] += 1
C:\Users\vivia\AppData\Local\Temp\ipykernel_5112\2708045039.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  FinalPlayOff[year] = FinalPlayOff[year].append(row)
C:\Users\vivia\AppData\Local\Temp\ipykernel_5112\2708045039.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_groups_data[year].Wins[winner_row] += 1
C:\Users\vivia\AppData\Local\Temp\ipykernel_5112\270804503

In [23]:
FinalPlayOff[2022]

KeyError: 2022